In [13]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *

In [14]:
def spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops,broad2=False):
    output_par = np.zeros((6,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    output_par_err = np.zeros((6,mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    fitted = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    residual = np.zeros((np.shape(mini_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])    
            popt_full_fit,pcov_full_fit = leastsq(full_gauss2_eline_fixkin,x0=[0.1, 0.1, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)
            
            output_par[:,i,j]=popt_full_fit
            fitted[:,i,j]=(full_gauss2_eline_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
            residual[:,i,j] = mini_cube[:,i,j] - fitted[:,i,j]
            
            spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
            for l in range(MC_loops):
                iteration_data = np.random.normal(spectrum,error)   
                popt_spec_MC,pcov_spec_MC = leastsq(full_gauss2_eline_fixkin,x0=[0.1, 0.1, 0.01, 0.01, -0.7, 0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
                spec_parameters_MC[:,l]=popt_spec_MC
                
            spec_parameters_err = np.std(spec_parameters_MC,1)
            output_par_err[:,i,j] = spec_parameters_err
            print (i,j)
    return output_par,output_par_err,fitted,residual

def flux_map(output_par,obj,destination_path_cube="/Volumes/Seagate/HE0040-1105 Extended"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(output_par[1,:,:],name='amp_OIII5007'))
    hdus.append(fits.ImageHDU(output_par[2,:,:],name='amp_OIII5007_br'))
    hdus.append(fits.ImageHDU(output_par[3,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(output_par[4,:,:],name='m'))
    hdus.append(fits.ImageHDU(output_par[5,:,:],name='c'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Maps/jetcloud_par_%s.fits'%(destination_path_cube,obj),overwrite='True')

def flux_map_err(output_par_err,obj,destination_path_cube="/Volumes/Seagate/HE0040-1105 Extended"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(output_par_err[0,:,:],name='amp_Hb_err'))
    hdus.append(fits.ImageHDU(output_par_err[1,:,:],name='amp_OIII5007_err'))
    hdus.append(fits.ImageHDU(output_par_err[2,:,:],name='amp_OIII5007_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[3,:,:],name='amp_Hb_br_err'))
    hdus.append(fits.ImageHDU(output_par_err[4,:,:],name='m_err'))
    hdus.append(fits.ImageHDU(output_par_err[5,:,:],name='c_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Error Maps/jetcloud_par_err_%s.fits'%(destination_path_cube,obj),overwrite='True')


In [15]:
def spectroastrometric_script(obj,z,MC_loops=100,destination_path_cube="/Volumes/Seagate/HE0040-1105 Extended"):
    fixed_param = fixed_parameters_residual_emission(obj)
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s.1_arcsec_minicube.fits'%(destination_path_cube,obj))
 
    (output_par,output_par_err,fitted_cube,residual_cube) = spectroastrometry(mini_cube,mini_err,mini_wave,fixed_param,MC_loops)
    
    store_cube('%s/%s.jetcloud_fitted_residual_minicube.fits'%(destination_path_cube,obj),fitted_cube,mini_wave,residual_cube,mini_header)    
    flux_map(output_par,obj)
    flux_map_err(output_par_err,obj)
    
    #(centroid_OIII_nr,centroid_OIII_br,centroid_Hb_blr_br) = light_weighted_centroid(obj)
    print ('%s.fits'%(obj))

In [16]:
z = {"HE0040-1105":0.041692}

objs = z.keys()

for obj in objs:
    spectroastrometric_script(obj,z[obj])

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
14 0
14 1
14 2
14 3
14 4
14 5
